In [1]:
import numpy as np
import tensorflow as tf
from copy import deepcopy

import os

import datetime

import import_ipynb
from QBert import train_utils, models

from tqdm.notebook import tqdm

importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\train_utils.ipynb
importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\models.ipynb


In [2]:
# sub_model (core_model 필요 Config)

vocab_size = 32000 # 
hidden_size = 768 # Transformer hidden Layers
type_vocab_size = 12 #: The number of types that the 'type_ids' input can take.
num_layers = 12
num_attention_heads = 12
max_seq_length = 256 # 512
dropout_rate = .1
# attention_dropout_rate = .1
inner_dim = 3072
# hidden_act = 'gelu'
initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)

# Pretrain Model 필요 Config
max_predictions_per_seq = 40


In [24]:
filenames = ['./Test_Examples.tfrecords' ]

# Create a description of the features.
feature_description = {
    'input_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'input_mask': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'masked_lm_positions': tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
    'masked_lm_ids': tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
    'masked_lm_weights': tf.io.FixedLenFeature([max_predictions_per_seq], tf.float32),
    'next_sentence_labels': tf.io.FixedLenFeature([1], tf.int64),
}

# keys = feature_description.keys()

In [4]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

def _select_data_from_record(record):
    """Filter out features to use for pretraining."""
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids'],
        'masked_lm_positions': record['masked_lm_positions'],
        'masked_lm_ids': record['masked_lm_ids'],
        'masked_lm_weights': record['masked_lm_weights'],
    }
    if use_next_sentence_label:
        x['next_sentence_labels'] = record['next_sentence_labels']
    if use_position_id:
        x['position_ids'] = record['position_ids']

    # TODO(hongkuny): Remove the fake labels after migrating bert pretraining.
    if output_fake_labels:
        return (x, record['masked_lm_weights'])
    else:
        return x


In [29]:

BUFFER_SIZE = 10000

GLOBAL_BATCH_SIZE = 4
#BATCH_SIZE_PER_REPLICA = np.ceil(GLOBAL_BATCH_SIZE // strategy.num_replicas_in_sync)

use_next_sentence_label = True
output_fake_labels = True
use_position_id = False


In [30]:


train_dataset = tf.data.Dataset.from_tensor_slices(filenames)
train_dataset = train_dataset.interleave(tf.data.TFRecordDataset, cycle_length = -1)

dataset_inputs = train_dataset.map(_parse_function) # String to Example
dataset_inputs_with_labels = dataset_inputs.map(_select_data_from_record) # Example to InputData
## 본래대로라면 그냥 써도 되지만, 현재 Label이 없는 데이터이기 때문에
## max_predictions_per_seq 길이의 허위 정답 (Fake_y)를 삽입하는 mapping function이다.

dataset = dataset_inputs_with_labels
dataset = dataset.cache()
dataset = dataset.shuffle(10000, reshuffle_each_iteration = True)
dataset = dataset.batch(GLOBAL_BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)



In [31]:
# optimizer

## Learning Rate Decay

# lr = 1e-4 warmup stage (step <= 10000)
# Decay linearly

init_lr = 1e-4
warmup_steps = 10000
num_train_steps = 20 * 50000
end_lr = 0

decay_schedule_fn = tf.keras.optimizers.schedules.PolynomialDecay(
      initial_learning_rate=init_lr,
      decay_steps=num_train_steps,
      end_learning_rate=end_lr)

lr_schedule = train_utils.WarmUp(
        initial_learning_rate=init_lr,
        decay_schedule_fn=decay_schedule_fn,
        warmup_steps=warmup_steps)

optimizer = train_utils.AdamWeightDecay( 
    learning_rate=lr_schedule,
    weight_decay_rate=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-6,
    exclude_from_weight_decay=['LayerNorm', 'layer_norm', 'bias'])

In [32]:
# loss
def loss_fn(fake_y, losses, **unused_args) :
    
    return tf.reduce_mean(losses, axis = -1)
    

In [38]:
model, sub_model = models.get_bert_models_fn(vocab_size
                                         , hidden_size
                                         , type_vocab_size
                                         , num_layers
                                         , num_attention_heads
                                         , max_seq_length
                                         , max_predictions_per_seq
                                         , dropout_rate
                                         , inner_dim 
                                         , initializer)
model.compile(optimizer, loss=loss_fn)

In [39]:
model.load_weights(filepath='./training_checkpoints_load/ckpt_1')

In [40]:
inputs, labels = next(iter(dataset))

In [47]:
sub_res = sub_model([inputs['input_ids'], inputs['input_mask'][:, tf.newaxis, tf.newaxis, :], inputs['segment_ids']])

In [90]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_mask (InputLayer)         [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_word_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_7 (Te [(None, 1, 1, 256)]  0           input_mask[0][0]                 
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 256)]        0                                            
____________________________________________________________________________________________

## NSP

In [109]:
?tf.keras.metrics.sparse_categorical_crossentropy

In [104]:
tf.argmax(tf.nn.softmax(tf.math.exp(model.layers[-2](sub_res['pooled_output'][0:1]))), axis = 1)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0], dtype=int64)>

In [107]:
inputs['next_sentence_labels'][0]

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0], dtype=int64)>

## MLM 

In [67]:
layer_x = model.layers[6]
layer_x._output_type = 'predicitions'

In [69]:
output_logits = layer_x([sub_res['sequence_output'], tf.cast(inputs['masked_lm_positions'], dtype = tf.int32)])

In [70]:
prediction = tf.math.exp(output_logits)

In [73]:
tf.argmax(prediction[0], axis = 1)

<tf.Tensor: shape=(40,), dtype=int64, numpy=
array([15, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       15, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 15, 17, 17, 17, 17,
       17, 17, 17, 17, 15, 15], dtype=int64)>

In [75]:
inputs['masked_lm_ids'][0]

<tf.Tensor: shape=(40,), dtype=int64, numpy=
array([   15, 26911,  7719, 14883,    15, 14161,  7126, 24650,    75,
        3146,  7966,    17,  3374,   134,  1779,  1779,  6353,  1779,
        3147, 16103,  3032,  5578,  1021,    11, 12982,  5596,  5777,
        2002,    11, 11153,  6797,  6797,  9634,  7832,  7729, 18880,
       18880,  5621,     0,     0], dtype=int64)>

In [77]:
from transformers import BertTokenizerFast
tokenizer_for_load = BertTokenizerFast.from_pretrained('./model/BertTokenizer-3000-32000-vocab.txt'
                                                   , strip_accents=False
                                                   , lowercase=False)

Calling BertTokenizerFast.from_pretrained() with the path to a single file or url is deprecated


In [84]:
print(tokenizer_for_load.convert_ids_to_tokens(tf.argmax(prediction[2], axis = 1)))

['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', ',', ',', ',']


In [86]:
print(tokenizer_for_load.convert_ids_to_tokens(inputs['masked_lm_ids'][2]))

['##부터', '밀란', '##10', '승리를', ',', '12', '보', '2월', '팔레', '차지하였다', '.', '2', '##데', '23일', 'uefa', '2003', '##에서', '부상으로', '122', '3', '기여하였다', '밀려', '##하였는데', '-', '##드', '뛰', '시즌에', '슈퍼컵', '##혔', '##차', ',', '시즌', '-', '##내며', '다비', '##코와', '##스와의', '활약', '[PAD]', '[PAD]']
